# Session 12

## Exercises

Analyze the `national_covid19_age_range.csv` dataset.

### 1. Statistics by sex

1. Exclude rows with age range "Total"
2. Create two separate `DataFrames`, one for women and one for men
3. Compute the median deceases, the mean ICU cases, and the standard deviation of ICU cases for the two sexes

### 2. Statistics by age

1. Create two separate `DataFrames`: one for all the ages except "Total", and one for ages 20 to 40
2. Compute the average percentage of hospitalized cases with respect to confirmed cases for the two groups.